# Ints, Floats, Doubles, and Error

W. Ross Morrow [ross@bond.tech](mailto:ross@bond.tech)

09/08/2020

# Introduction

Here we analyze the impact of datatypes on basic operations, accumulation with addition and subtraction, of data we intend to represent monetary quantities. We execute calculations with `int`s, `long`s, `float`s, and `double`s that represent what might be reflective of totalling up transactions and payments, and study how the number of amounts to aggregate impacts accuracy. We don't consider operations, like taking taxes or interest using percents, that would either _require_ floating point or perfect rational math. For now, we keep it simple and limited to adding things up and ask mainly about "how many".  

Specifically our goal is to investigate if and when floating point numbers can accurately reflect the true results of such calculations, in cents. We pack `int`s in too, because there's a general question about the space required to store the results of the calculations we do. In this work we don't care whether the floating point representations themselves, in all their precision, match a true value in cents. Spoiler: they won't. We only care that those floating point values, _when rounded to cents_ , _then_ match the true value. This is possible with `double`s, but not `float`s, although depending on the size of the accumulations we might need to use specific summation methods for `double`s (that aren't at all hard to program). If that holds, then we have some assurance that `double` datatypes can serve as an effective intermediary, at least: if we can persist `long int` data, manipulate it with `double`s (so long as this is convenient), and store it back as `long int` _correctly_ then we basically never lose precision _in the units we care about_ (here presumed to be cents). 

Our numerical experiments suggest `double`s are fine in this lens for sums of hundreds of thousands or even millions of actually pretty large amounts. Our experimental analysis is, obviously and inevitably, dependent on the assumptions we make in formulating it. We also undertake a theoretical analysis that provides _guarantees_ that `double`s are fine, under our assumptions for drawing amounts, for aggregations of tens of thousands of amounts. If we're careful about how we sum amounts up, guaranteed accuracy of aggregations extends to the millions of amounts. And all under kind of silly assumptions that generate rather large amounts which, if relaxed, would further increase the number of amounts we could aggregate with a guarantee of correctness in cents. 

A super important point to note is that, these days, "`double` is often the default". In particular, [`python` floats are doubles](https://docs.python.org/3/tutorial/floatingpoint.html) (unless you say otherwise), and I've even heard that "under the hood" single-precision `float` calculations are done in 64B (`double`) hardware (and I would love either a correction or a reference). We have to explicitly store `double` data in our databases to keep the precision, though this is really more about storing 8 bytes rather than 4. Point is that while amounts are being used it's probably not an awful mistake to just presume some automagic has put double precision to work for you. And definitely never use single precision. 

We also review some basics about floating point representation, including spooky edge cases, but mainly to mediate those with results guaranteeing precision in our particular case. More or less, the takeaway here is use 8 bytes (8B) in either `long int` or `double` form and you're good. But _nothing's ever so simple as that kind of soundbite_ , and I hope this article also displays some of the exciting complexities involved here. 

# Initial Experiments

We start with some basic imports, as is common. 

In [46]:
import pandas as pd
import numpy as np

We use a `C` code, `draft.c`, to run some tests quickly and be sure of the bytewidth of different datatypes. `draft`  accepts three arguments in this order: an accumulation size ($N$), a(n optional) number of trials ($T$), and an (optional) filename for the results. The call below ignores the third argument, and runs $10,000$ trials of $100,000$ element accumulations with addition, subtraction, or both. For each trial, `draft` draws $N$ random amounts in the range 
$$
\begin{aligned}
\\
    [\; 0 \; , \; 2147483647 \; ] \mod 10000000
\\
\\
\end{aligned}
$$
This is hacky, but a reasonably uniform distribution of values up to about \$100,000 as shown in the next cell. 

In [68]:
vs = np.mod( np.random.randint(0,2147483648,1000000,dtype=np.int) , 10000000 )
hg = np.histogram( vs , bins=1000 )
print( f"Drawn values are in [{hg[1].min()/100.0},{hg[1].max()/100.0}] (dollars)" )
print( f"  typically with {hg[0].mean()} +/- {1.96*hg[0].std()} values in 1000 bins" )

Drawn values are in [0.03,99999.81] (dollars)
  typically with 1000.0 +/- 63.52232698508455 values in 1000 bins


Each amount roughly represents a "transaction" value. We choose \$100,000 just to explore extremes, as this is where roundoff error will show it's ugly face if any. Put another way, we need sample data amounts with very different orders of magnitude to have any hope of revealing issues with a loss of significant digits. I haven't tried yet, but if we're working with 100's of dollar amounts we just won't see anything interesting for the computations impatience will allow us to undertake. 

If you want to change this setting though, just change 
```
#define MAX_CENTS 10000000
```
Again, this distribution has unrealistically large "typical" values for amounts, but gives us a chance to explore extremes in the hop of finding flaws in modeling amounts with various datatypes. We should probably use something like a log normal or power law distribution that is most likely to have small amounts, but can still have rather large draws. 

With each such random value, (w.l.o.g.) drawn as a 32-bit (4B) `int` in cents, we also store 
* the same value cast "up" to a `long` (8B) `int`
* the same value cast to a 32-bit (4B) `float` in _both_ dollars and cents
* the same value cast to a 64-bit (8B) `double` in _both_ dollars and cents

For repeated addition and subtraction operations, the `long int` version gives us the ground truth. The maximum value (9,223,372,036,854,775,807) is so large we would need to run an accumulation of over 922 _trillion_ values to overflow. This is larger even than the largest (4B) `int`, which is our input type for $N$, so we completely ignore this. The code wouldn't run anyway; storing an "amount" in our sense costs 36B, and 900T amounts would take something like 31,000GB of RAM. To run in under 1GB of memory we need about $N \leq 29,826,162$ which still represents a rather large (daily) amount accumulation. For a bit of [reference](https://www.statista.com/statistics/261327/number-of-per-card-credit-card-transactions-worldwide-by-brand-as-of-2011/), in 2019 worldwide, Visa and Mastercard together had on average 401,369,863 transactions a day worldwide, and Amex had about 21,917,808 transactions a day. And, interpreting `long`s as values in cents, we can store well over 900k Bezos' (1 Bezos ~ 100 billion dollars) in monetary value. 

With a set of $N$ amounts stored in these different types and precisions (dollars and cents for the `float`s and `double`s), we do the following three things: 
* add them all up into one total number
* subtract them all into one total number
* add or subtract each number from a total, based on a fair coin flip (50\% probability of either)

For each of these operations we do them all with each type and precision. Again, the `long int` (in cents) will be exact. The `int` versions will very likely overflow for seemingly reasonable numbers of amounts. The `float` and `double` ones are up in the air. 

The code writes a `csv` file, named `results.csv` by default, with a line for each trial using distinct random amounts. There are columns for each non-exact value -- `int` cents, `float` cents, `float` dollars, `double` cents, and `double` dollars --  with a `0` if the result is wrong (relative to `long int`) and `1` if it is correct. 

If you need/want to compile the code, run both the following cells. If you just want to run the code, just run the second cell. You only need to compile once, so long as you don't change the code in `draft.c`. You need to recompile for any change. The run below is for $N = 100,000$. 

In [234]:
!gcc -O3 draft.c -o draft

In [121]:
!./draft 100000 10000 results-100000-10000.csv

Now that we have a `results.csv` (or whatever) file, we can read it in as a `DataFrame` and summarize the fractions of trials in which each type/precision resulted in a correct calculation. This is done below, where the column names amount to first-letter abbreviations (`f_c` is `float`, cents). The `mean` and `std` just aggregate the `1`s and `0`s in the columns for a quick look. Actually, our trials are a Bernoulli process so the `mean` of the outcomes is the probability $p$ of being "correct", and `std` should be $\sqrt{p(1-p)}$ (which seems about right in the data I've seen). 

In [122]:
df = pd.read_csv( 'results-100000-10000.csv' , header=None , names=['trial','op','i_c','f_c','f_d','d_c','d_d'] )
df[['op','i_c','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

i_c               f_c        f_d       d_c       d_d     
      mean       std    mean   std mean  std mean  std mean  std
op                                                              
+   0.0000  0.000000  0.0000  0.00    0  0.0    1  0.0    1  0.0
-   0.0000  0.000000  0.0000  0.00    0  0.0    1  0.0    1  0.0
?   0.7577  0.428496  0.0001  0.01    0  0.0    1  0.0    1  0.0

I've run this a bunch of times and get results like above. `float`s are always wrong, `int`s are sometimes right with mixed adds/subtract but never right with just adds and subtracts, and `double`s are always right. 

These results depend on the accumulation size ($N$, the first argument to `draft`). In the cell below we run again, with $N = 100$. 

In [123]:
!./draft 100 10000 results-100-10000.csv
df = pd.read_csv( 'results-100-10000.csv' , header=None , names=['trial','op','i_c','f_c','f_d','d_c','d_d'] )
df[['op','i_c','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

i_c          f_c               f_d            d_c       d_d     
   mean  std    mean       std    mean       std mean  std mean  std
op                                                                  
+     1  0.0  0.0062  0.078500  0.0042  0.064674    1  0.0    1  0.0
-     1  0.0  0.0062  0.078500  0.0042  0.064674    1  0.0    1  0.0
?     1  0.0  0.0482  0.214199  0.0532  0.224443    1  0.0    1  0.0

In [236]:
!./draft 1000 10000 results-1000-10000.csv
df = pd.read_csv( 'results-1000-10000.csv' , header=None , names=['trial','op','i_c','f_c','f_d','d_c','d_d'] )
df[['op','i_c','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

i_c          f_c               f_d            d_c       d_d     
   mean  std    mean       std    mean       std mean  std mean  std
op                                                                  
+     0  0.0  0.0002  0.014141  0.0001  0.010000    1  0.0    1  0.0
-     0  0.0  0.0002  0.014141  0.0001  0.010000    1  0.0    1  0.0
?     1  0.0  0.0056  0.074627  0.0053  0.072612    1  0.0    1  0.0

In [240]:
!./draft 10000 10000 results-10000-10000.csv
df = pd.read_csv( 'results-10000-10000.csv' , header=None , names=['trial','op','i_c','f_c','f_d','d_c','d_d'] )
df[['op','i_c','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

i_c          f_c               f_d            d_c       d_d     
   mean  std    mean       std    mean       std mean  std mean  std
op                                                                  
+     0  0.0  0.0000  0.000000  0.0000  0.000000    1  0.0    1  0.0
-     0  0.0  0.0000  0.000000  0.0000  0.000000    1  0.0    1  0.0
?     1  0.0  0.0005  0.022356  0.0002  0.014141    1  0.0    1  0.0

Here `int`s are always right, `float`s are sometimes (but rarely) right, and again `double`s are never wrong. (Never use single precision.)

What about even larger accumulations? Like a million? 

In [124]:
!./draft 1000000 10000 results-1000000-10000.csv
df = pd.read_csv( 'results-1000000-10000.csv' , header=None , names=['trial','op','i_c','f_c','f_d','d_c','d_d'] )
df[['op','i_c','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

i_c            f_c       f_d       d_c          d_d         
      mean       std mean  std mean  std mean  std    mean      std
op                                                                 
+   0.0000  0.000000    0  0.0    0  0.0    1  0.0  0.9993  0.02645
-   0.0000  0.000000    0  0.0    0  0.0    1  0.0  0.9993  0.02645
?   0.2908  0.454154    0  0.0    0  0.0    1  0.0  1.0000  0.00000

Finally something interesting! With 1M amounts to accumulate, `double`s _in dollars_ are correct only 99.9% of the time if we are exclusively adding or exclusively subtracting positive numbers (from our naively chosen distribution, of course). If we're mixing operations, or using `double`s _in cents_ , things are still fine. 

What about 10M? **Caution** running this takes time. 

In [119]:
!./draft 10000000 10000 results-10000000-10000.csv
df = pd.read_csv( 'results-10000000-10000.csv' , header=None , names=['trial','op','i_c','f_c','f_d','d_c','d_d'] )
df[['op','i_c','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

i_c            f_c       f_d       d_c          d_d          
      mean       std mean  std mean  std mean  std    mean       std
op                                                                  
+   0.0000  0.000000    0  0.0    0  0.0    1  0.0  0.1087  0.311278
-   0.0000  0.000000    0  0.0    0  0.0    1  0.0  0.1087  0.311278
?   0.0983  0.297735    0  0.0    0  0.0    1  0.0  1.0000  0.000000

So with 10M numbers to accumulate, for pure addition and subtraction accumulations and `double`s in dollars, we see really bad performance: correctness only 10% of the time. But for mixed operations or `double`s in cents, using `double`s is still always correct. 

# What are Floating-Point Numbers Anyway? 

We aren't really going to give a comprehensive explanation of these results, but we will provide hints and analysis. And to do that we need to review floating point math. Besides, it's wild. 

[IEEE 754](https://en.wikipedia.org/wiki/IEEE_754-2008_revision) `double`s store real numbers $x$ essentially as tuples 
$$
\begin{aligned}
\\
    (s(x),b(x),e(x)) \in \{0,1\} \times \{0,1\}^{52} \times [0,2047]
\\
\\
\end{aligned}
$$
where
$$
\begin{aligned}
\\
    \texttt{double}(x) = (-1)^{s(x)}\left( 1 + \sum_{k=1}^{52} \frac{b_{52-k}(x)}{2^k} \right) 2^{e(x)-1023}
\\
\\
\end{aligned}
$$
The "first" bit is a sign bit, the next $11$ are a base-2 exponent (read as an unsigned int), and the last $52$ are base-2 decimals (the "fractional part" or "significand"). 

Here are some rough examples: 
* $\texttt{fl}(1)$: $s(x) = 0$, $e(x) = 1023$, and $b = 0$. 
* $\texttt{fl}(-1)$: $s(x) = 1$, $e(x) = 1023$, and $b = 0$. 
* $\texttt{fl}(10)$: $s(x) = 0$, $e(x) = 1026$, and $b \approx 0.25$
* $\texttt{fl}(100)$: $s(x) = 0$, $e(x) = 1029$, and $b \approx 0.5625$. 

Here are some _exact_ examples: 

* $\texttt{fl}(1)$: $s(x) = 0$, $e(x) = 1023$, $b_{51:0} = \texttt{000000000000000000000000000000000000000000000000}$
* $\texttt{fl}(-1)$: $s(x) = 1$, $e(x) = 1023$, $b_{51:0} = \texttt{000000000000000000000000000000000000000000000000}$
* $\texttt{fl}(0.5)$: $s(x) = 0$, $e(x) = 1022$, $b_{51:0} = \texttt{000000000000000000000000000000000000000000000000}$
* $\texttt{fl}(10)$: $s(x) = 0$, $e(x) = 1026$, $b_{51:0} = \texttt{010000000000000000000000000000000000000000000000}$
* $\texttt{fl}(100)$: $s(x) = 0$, $e(x) = 1029$, $b_{51:0} = \texttt{100100000000000000000000000000000000000000000000}$
* $\texttt{fl}(0.01)$: $s(x) = 0$, $e(x) = 1016$, $b_{51:0} = \texttt{010001111010111000010100011110101110000101000111}$
* $\texttt{fl}(\pi)$: $s(x) = 0$, $e(x) = 1024$, $b_{51:0} = \texttt{100100100001111110110101010001000100001011010001}$

Note I use a hacky notation $b_{51:0}$ to denote that the bitstrings are actually printed in reverse. Sorry, it was alot easier this way. 

Let's look carefully at one of these in particular, $100$. The above translates to 
$$
\begin{aligned}
\\
    \texttt{double}(100) 
        = (-1)^{0}\left( 1 + \frac{1}{2} + \frac{1}{2^4} \right) 2^{6}
        = \left( 1 + \frac{1}{2} + \frac{1}{16} \right) 64
        = \left( 1 + 0.5 + 0.0625 \right) 64
        = 1.5625 \times 64 = 100
\\
\\
\end{aligned}
$$
While the formulas and the data look insane, this isn't so bad, right? 

The exact representations come from the code `plotbits.c`, which you can read and run to get a better sense of how floating point numbers are represented (at least in the Intel chips used by Mac OSX and Apple `clang` version `11.0.3`). You can change values in `main` to print the representation of different numbers, like in the call
```
print_double_bits( M_PI );
```
which prints `math.h`'s defined $\pi$ value as
```
(d) 3.1415926535897931:
    sign... 0
    exp.... 1024 (00000100|00000000)
    bits... 10010010|00011111|10110101|01000100|01000010|11010001
    full... 0,1000000|0000,1001|00100001|11111011|01010100|01000100|00101101|00011000
```
Here the `|` characters separate bytes (B) in multi-byte datatypes. In the `full` line of the print we also use `,` to separate _logical_ fields of the `double` representation. At the very least, reviewing this may give an appreciation of the startling complexity (and ingenuity!) of effectively representing real numbers in computers. 

### Doubles in Cents

Let's make sure that a 32-bit integer $i$, in cents, can be represented exactly _up to cents_ in a `double`. (Recall we presume we can store any given amount, in cents, using a standard `int`, but need `long int`s to store reasonable accumulations of such amounts.) This way we at least know that we can store exact values without _relevant_ precision loss in a `double`. If we can't do that, we should probably give up. (Spoiler 2: we can.)

It is actually quite trivial to show that an `int` has an exact `double` representation. Obviously our integer $i$ has a unique base-2 representation: 
$$
\begin{aligned}
\\
    \texttt{int}(i) 
        &= (-1)^{s(i)}\left( \sum_{k=1}^{31} b_{k} 2^{k-1} \right)
            = (-1)^{s(i)}\left( \sum_{k=1}^{31} b_{k} 2^{k-31} \right) 2^{30}
            = (-1)^{s(i)}\left( \sum_{k=0}^{30} \frac{b_{31-k}}{2^{k}} \right) 2^{30}
\\
\\
\end{aligned}
$$
More or less this is what is stored as the `int` value. Now if $b_{31} = 1$, we have 
$$
\begin{aligned}
\\
    \texttt{int}(i) 
        &= (-1)^{s(i)}\left( 1 + \sum_{k=1}^{30} \frac{b_{31-k}}{2^{k}} \right) 2^{30}
\\
\\
\end{aligned}
$$
which is in `double` representation $(s(i),b^\prime,1053)$ with fractional part bits $b^\prime$ satisfying
$$
\begin{aligned}
\\
    b^\prime_{52-k} = b_{31-k} \;\;,\;\; k = 1,\dotsc,30
    \quad\text{and}\quad
    b^\prime_{52-k} = 0 \;\;,\;\; k = 31,\dotsc,52
\\
\\
\end{aligned}
$$
Otherwise, let $b_{31} = \dotsb = b_{31-k_0-1} = 0$, $b_{31-k_0} = 1$; then 
$$
\begin{aligned}
\\
    \texttt{int}(i) 
        &= (-1)^{s(i)}\left( \sum_{k=k_0}^{30} \frac{b_{31-k}}{2^{k}} \right) 2^{30} 
        = (-1)^{s(i)}\left( \sum_{k=k_0}^{30} \frac{b_{31-k}}{2^{k-k_0}} \right) 2^{30-k_0} \\
        &= (-1)^{s(i)}\left( 1 + \sum_{k=k_0+1}^{30} \frac{b_{31-k}}{2^{k-k_0}} \right) 2^{30-k_0}
        = (-1)^{s(i)}\left( 1 + \sum_{k=1}^{30-k_0} \frac{b_{31-k+k_0}}{2^{k}} \right) 2^{30-k_0} \\
\\
\\
\end{aligned}
$$
which is in `double` representation $(s(i),b^\prime,1053-k_0)$ with fractional part bits $b^\prime$ satisfying
$$
\begin{aligned}
\\
    b^\prime_{52-k} = b_{31-k+k_0} \;\;,\;\; k = 1,\dotsc,30-k_0
    \quad\text{and}\quad
    b^\prime_{52-k} = 0 \;\;,\;\; k = 30-k_0+1,\dotsc,52
\\
\\
\end{aligned}
$$

So, roughly speaking, by absorbing the number of "leading" zero bits into the exponent and (possibly) reording the indexing of the `int` bits into the `double`'s fractional part, we have an exact representation of the `int` as a `double`. (Note that single-precision `float`s only have fractional parts with 23 bits, meaning we would have to truncate the bit sum in the above leading to casting error when the integers have nonzero leading bits and thus are large.)

If we keep our `double` values in cents, that's it for one direction: taking an `int` and putting it in a `double`. If we keep our `double` values in dollars, we have to also divide by `100.0` which will introduce some extra error. We deal with that later. 

Now we can think about the second part: taking the `double` back to an `int`. If we are using dollars, we have to multiply by `100.0` to get cents, then convert. But let's first consider `double`s storing cents. To convert a `double` to an `int` we have to think a bit about _how_ we want to do this. A naive typecast `(int)double` conversion will _truncate_ the double, disregarding anything past the decimal (the fractional part) in a base-10 representation; a cast `(int)round(double)` conversion will round up or down based on the fractional part, obtaining a pure integer `double` representation, and then cast that. 

The naive typecast is where we can possibly get into trouble and where we have canonical (but edge case) examples like 
```
(int)(0.9999999999999999) = 0
```
Rounding isn't exactly "safe" either because 
```
(int)round(0.5) = 1
(int)round(0.4999999999999999) = 0
```

While those examples sound scary, let's think it through in light of the casting derivation above. Admitting $k_0 = 0$ we have the general formula
$$
\begin{aligned}
\\
    \texttt{double}(\texttt{int}(i)) 
        = (-1)^{s}\left( 1 + \sum_{k=1}^{30-k_0} \frac{b_{31-k+k_0}}{2^{k}} \right) 2^{30-k_0}
\\
\\
\end{aligned}
$$
What is this number's decimal part? If we literally multiply by the power of two, we have
$$
\begin{aligned}
\\
    \texttt{double}(\texttt{int}(i)) 
        &= (-1)^{s}\left( 2^{30-k_0} + \sum_{k=1}^{30-k_0} \frac{b_{31-k+k_0}}{2^{k}} 2^{30-k_0} \right)
        = (-1)^{s}\left( 2^{30-k_0} + \sum_{k=1}^{30-k_0} \frac{b_{31-k+k_0}}{2^{k+k_0-30}} \right)
\\
\\
\end{aligned}
$$
We can then ask: when are the exponents in the denominator positive? For any decimal part can't come from _non-positive_ (negative or zero) powers of two in the denominator, those are integers in the sum. But the denominator powers are positive if, and only if, $k > 30-k_0$ and our sum only goes up to $30-k_0$; so _there are no positive exponents_. Thus the sum _has no fractional part_ in real number terms, being a sum of integers, and _both_ truncation and rounding will give exact answers. In other very mathy words:
$$
\begin{aligned}
\\
    \texttt{int}(\texttt{double}(\texttt{int}(i))) = \texttt{int}(i)
\\
\\
\end{aligned}
$$
or $\texttt{int}$ (either truncation or rounding) is a left-inverse of $\texttt{double}$ _on the range of_ $\texttt{int}$ in the (8B) floating-point field. 

What exactly are we saying here? We just said, with our edge cases, that "$\texttt{double}\to\texttt{int}$ is noisy", but then right away we said "$\texttt{double}\to\texttt{int}$ is exact". This is not a contradiction. What we've said in our two steps is that if we convert an `int` to `double` we get an exact `int` in the `double`'s language and can convert it right back to an `int`; we _never_ get the funky edge case expansions with weird decimal place expansions that cause trouble when trying to write back to an `int`. (That's what left inverse _on the range of_ means.) If we read an `int` into a `double`, we can read it right back out. That's the first thing we wanted to show, that we can at least represent amounts in cents using `double`s without _any_ error. 

So are those truncation and rounding rules never relevant? Of course not. Those become relevant as soon as we start doing floating point math with a `double` converted from an `int`. In math, we can't be sure the following holds: 
$$
\begin{aligned}
\\
    \texttt{int}(\; \texttt{f}(\; \texttt{double}(\texttt{int}(i)) \; )\; ) = \texttt{int}(i)
\\
\\
\end{aligned}
$$
where $\texttt{f}$ is some program that operates on `double`s. We'll get to more of that soon. 

So when do `double`s cast exactly to `int`s? Let's take a generic `double`: 
$$
\begin{aligned}
\\
    (-1)^{s}\left( 1 + \sum_{k=1}^{52} \frac{b_{52-k}}{2^k} \right) 2^{e-1023}
\\
\\
\end{aligned}
$$
Clearly the sign $s$ carries over directly, and we must have $0 \leq e-1023 < 30$ to (a) not have a pure decimal (a number less than one in magnitude) and (b) a representable `int`. Let $E = e-1023 \in [0,30)$. Then 
$$
\begin{aligned}
\\
     2^{E} + \sum_{k=1}^{52} b_{52-k}2^{E-k}
         &= \left( 2^{E} + \sum_{k=1}^{E} b_{52-k}2^{E-k} \right) 
             + \left( \sum_{k=E+1}^{52} \frac{b_{52-k}}{2^{k-E}} \right) \\
         &= \left( 2^{E} + \sum_{k=0}^{E-1} b_{(52-E)+k}2^{k} \right) 
             + \left( \sum_{k=1}^{52-E} \frac{b_{(52-E)-k}}{2^{k}} \right)
\\
\\
\end{aligned}
$$
the first quantity in parentheses is an integer, the second a decimal. If we were to truncate the `double` (i.e., take the `floor`), we would just forget about 
$$
\begin{aligned}
\\
     b_{51-E} \;\; , \;\; 
     b_{50-E} \;\; , \;\; 
     \dotsb \;\; , \;\; 
     b_0
\\
\\
\end{aligned}
$$
Clearly, the _smaller_ $E$ is, the more bits of the significand we ignore. Note also that if
$$
\begin{aligned}
\\
     b_{51-E} = b_{50-E} = \dotsb = b_0 = 0
\\
\\
\end{aligned}
$$
then the number _is_ an integer. (This is a less stringent requirement for larger $E$, more strigent for smaller $E$.) 

Which `double`s, multiplied by 100, are integers? 
$$
\begin{aligned}
\\
    &\left( 1 + \frac{1}{2} + \frac{1}{2^4} \right)
            \left( 1 + \sum_{k=1}^{52} \frac{b_{52-k}}{2^k} \right) 
            2^{e-1017} \\
    &\quad\quad\quad\quad
    = \left( 1 + \frac{1}{2} + \frac{1}{2^4} 
                + \sum_{k=1}^{52} \frac{b_{52-k}}{2^k}
                + \sum_{k=1}^{52} \frac{b_{52-k}}{2^{k+1}} 
                + \sum_{k=1}^{52} \frac{b_{52-k}}{2^{k+4}} 
            \right)
            2^{e-1017} \\
    &\quad\quad\quad\quad
    = \left( 1 + \frac{1}{2} + \frac{1}{2^4} 
                + \sum_{k=1}^{52} \frac{b_{52-k}}{2^k}
                + \sum_{k=2}^{53} \frac{b_{53-k}}{2^{k}} 
                + \sum_{k=5}^{56} \frac{b_{56-k}}{2^{k}} 
            \right)
            2^{e-1017} \\
    &\quad\quad\quad\quad
    = \left( 1 + \frac{1+b_{51}}{2} + \frac{1}{2^4} 
                + \sum_{k=2}^{52} \frac{b_{52-k}+b_{53-k}}{2^k}
                + \sum_{k=5}^{56} \frac{b_{56-k}}{2^{k}} 
                + \frac{b_{0}}{2^{53}}
            \right)
            2^{e-1017} \\
    &\quad\quad\quad\quad
    = \left( 1 + \frac{1+b_{51}}{2}
                + \frac{b_{50}+b_{51}}{2^2}
                + \frac{b_{49}+b_{50}}{2^3}
                + \frac{1+b_{48}+b_{49}}{2^4}
                + \sum_{k=5}^{52} \frac{b_{52-k}+b_{53-k}+b_{56-k}}{2^k}
                + \frac{b_{0}+b_{3}}{2^{53}} 
                + \frac{b_{2}}{2^{54}} 
                + \frac{b_{1}}{2^{55}} 
                + \frac{b_{0}}{2^{56}} 
            \right)
            2^{e-1017}
\\
\\
\end{aligned}
$$

If $b_{51} = 0$, 
$$
\begin{aligned}
\\
    &\left( 1 + \frac{1}{2} + \frac{1}{2^4} \right)
            \left( 1 + \sum_{k=1}^{52} \frac{b_{52-k}}{2^k} \right) 
            2^{e-1017} \\
    &\quad\quad\quad\quad
    = \left( 
                1 + \frac{1}{2}
                    + \frac{b_{50}}{2^2}
                    + \frac{b_{49}+b_{50}}{2^3}
                    + \frac{1+b_{48}+b_{49}}{2^4}
                    + \sum_{k=5}^{52} \frac{b_{52-k}+b_{53-k}+b_{56-k}}{2^k}
                    + \frac{b_{0}+b_{3}}{2^{53}} 
                    + \frac{b_{2}}{2^{54}} 
                    + \frac{b_{1}}{2^{55}} 
                    + \frac{b_{0}}{2^{56}} 
            \right)
            2^{e-1017} \\
    &\quad\quad\quad\quad
    \overset{T}{=} \left( 
                1 + \frac{1}{2}
                    + \frac{b_{50}}{2^2}
                    + \frac{b_{49}+b_{50}}{2^3}
                    + \frac{1+b_{48}+b_{49}}{2^4}
                    + \sum_{k=5}^{52} \frac{b_{52-k}+b_{53-k}+b_{56-k}}{2^k}
            \right)
            2^{e-1017}
\\
\\
\end{aligned}
$$
where the last step ($\overset{T}{=}$) represents truncation of the bits a double won't be able to represent. (We can truncate these off because none of the powers of two in the sum can be factored out.)

If $b_{51} = 1$, 
$$
\begin{aligned}
\\
    &\left( 1 + \frac{1}{2} + \frac{1}{2^4} \right)
            \left( 1 + \sum_{k=1}^{52} \frac{b_{52-k}}{2^k} \right) 
            2^{e-1017} \\
    &\quad\quad\quad\quad
    = \left( 2 + \frac{1+b_{50}}{2^2}
                + \frac{b_{49}+b_{50}}{2^3}
                + \frac{1+b_{48}+b_{49}}{2^4}
                + \sum_{k=5}^{52} \frac{b_{52-k}+b_{53-k}+b_{56-k}}{2^k}
                + \frac{b_{0}+b_{3}}{2^{53}} 
                + \frac{b_{2}}{2^{54}} 
                + \frac{b_{1}}{2^{55}} 
                + \frac{b_{0}}{2^{56}} 
            \right)
            2^{e-1017} \\
    &\quad\quad\quad\quad
    = \left( 1 + \frac{1+b_{50}}{2^3}
                + \frac{b_{49}+b_{50}}{2^4}
                + \frac{1+b_{48}+b_{49}}{2^5}
                + \sum_{k=5}^{52} \frac{b_{52-k}+b_{53-k}+b_{56-k}}{2^{k+1}}
                + \frac{b_{0}+b_{3}}{2^{54}} 
                + \frac{b_{2}}{2^{55}} 
                + \frac{b_{1}}{2^{56}} 
                + \frac{b_{0}}{2^{57}} 
            \right)
            2^{e-1016} \\
\\
\\
\end{aligned}
$$
Presuming no new factors of $2$ can be extracted, the exponent $e^\prime$ is 
$$
\begin{aligned}
\\
    e^\prime - 1023 = e - 1016
    \quad\iff\quad
    e^\prime = e + 7
\\
\\
\end{aligned}
$$
More to the point, $E^\prime = e - 1016 \in [0,30)$ iff $e \in [1016,1046)$ and 
$$
\begin{aligned}
\\
     b_{1067 - e}^\prime = b_{1066 - e}^\prime = \dotsb = b_0^\prime = 0
\\
\\
\end{aligned}
$$

# Real Analysis (ha ha) 

The examples above aren't real analysis, in the sense that they are only examples. Can we do any analysis? 

There's a great, but very dense, book on floating point errors and numerous mathematical methods called [Accuracy and Stability of Numerical Algorithms](https://www.google.com/books/edition/Accuracy_and_Stability_of_Numerical_Algo/5tv3HdF-0N8C?hl=en&gbpv=1&printsec=frontcover) by Nick Higham. Chapter 2 presents a model of relative errors with floating point arithmetic: 
$$
\begin{aligned}
\\
    \texttt{fl}(x \texttt{op} y) = ( x \texttt{op} y ) ( 1 + \delta )
    \quad\quad
    |\delta| \leq u \approx 10^{-16}
    \quad\quad
    \texttt{op} = +,-,\times,/
\\
\\
\end{aligned}
$$
Here $u$ is the _unit roundoff_ ("the gap between numbers" from 30,000 feet). For IEEE floating point `double`s, $u \approx 10^{-16}$ (see Eqn. 4.4 on page 82). This model holds for IEEE 754 math, and is very useful. Chapter 4 covers summation, and for naive "recursive summation" (as implemented in `draft.c`) presents a weak error bound
$$
\begin{aligned}
\\
    \left| \sum_n x_n - \texttt{fl}\left(\sum_n x_n\right) \right| = | E_N | \leq (N-1)u \sum_n | x_n | + O(u^2)
\\
\\
\end{aligned}
$$
where $\texttt{fl}$ is the floating-point version of the summation, $E_N$ is the error from summing $N$ numbers $x_n$ and $u$ is again the _unit roundoff_. Note that because it references only the _signs_ of the numbers summed, this bound applies to addition, subtraction, or any mix of the two for the same summand magnitudes. 

Oh: and if you haven't heard the term "weak" before in reference to a bound, it doesn't mean "sometimes applies" but rather means "probably larger above the actual value across the relevant cases than we hope we could get." A "weak" bound is a bound, just one that mathy folks might expect improvement on or be generally dissatisfied with. Indeed, this bound can be improved with more specific summation methods, like the _pairwise summation_ we consider later, but we'll still take a look at what it says about computing accuracy. 

First, though, let's actually address storing doubles as dollars, where we can use the first model and bound. 

### Doubles in Dollars

Let's think instead of `double`s in dollars, instead of cents. In this frame we can think about some of the basic operations that might deleteriously affect precision. Given an integer $i$ representing an amount in cents, we've shown $\texttt{double}(\texttt{int}(i))$ is exact. Now what happens when we convert to dollars by dividing by $\texttt{double}(100.0)$? How do we get that in floating point terms? What about multiplying by $\texttt{double}(100.0)$, which we'll have to do to get cents back from a value in dollars? 

All the literal operations here are complicated and messy. The code `compare_conversions.c` explores when the transform
$$
\begin{aligned}
\\
    \texttt{int}\Bigg(\texttt{double}\Big(100.0\times\texttt{double}\big(\texttt{double}(i)/100.0)\big)\Big)\Bigg) 
        \neq \texttt{int}(i)
\\
\\
\end{aligned}
$$
(where the outermost $\texttt{int}$ operator is the simple bit conversion outlined above) is _byte_ exact rather than just _cents_ exact. 

In [230]:
!gcc compare_conversions.c -o compare

In [231]:
!./compare 

2147483647, 1.00
1846835937/2147483647 (86.00%) bytes match
2147483647/2147483647 (100.00%) correct


In other words: for 100% of the non-negative integers representable in 32 (signed) bits, we can _round_ to the correct value when converting to and from dollars; but only 86% of the conversions $\texttt{fl}(100.0\texttt{fl}(x_n/100.0))$ result in the same _bytes_ in the resulting `double` as the (exact) `int` originally cast into a `double`. 

One easy example is $7$: The literal `int` conversion is
```
(d) 7.0000000000000000:
    sign... 0
    exp.... 1025 (00000100|00000001)
    bits... 11000000|00000000|00000000|00000000|00000000|00000000
    full... 0,1000000|0001,1100|00000000|00000000|00000000|00000000|00000000|00000000
```
but $\texttt{fl}(100.0\texttt{fl}(7.0/100.0))$ is
```
(d) 7.0000000000000009:
    sign... 0
    exp.... 1025 (00000100|00000001)
    bits... 11000000|00000000|00000000|00000000|00000000|00000000
    full... 0,1000000|0001,1100|00000000|00000000|00000000|00000000|00000000|00000001
```
Note the literal bit mismatch in the least significant bit. This is inconsequential from the perspective of a valid cents place value using rounding, but shows that even unit conversions can introduce _some_ error. 

So let's look at that bound from above, starting with multiplication by 100. It's good to practice here anyway. Literally, it says
$$
\begin{aligned}
\\
    \texttt{fl}(100x) = 100x ( 1 + \delta )
    \quad\quad
    |\delta| \leq u \approx 10^{-16}
\\
\\
\end{aligned}
$$
Let's rearrange to get the magnitude of the literal error on one side: 
$$
\begin{aligned}
\\
    \texttt{fl}(100x) 
        &= 100x ( 1 + \delta ) = 100x + 100 x \delta \\
    \texttt{fl}(100x) - 100x 
        &= 100 x \delta \\
    | \texttt{fl}(100x) - 100x |
        &= 100 |x|| \delta | \leq |x| 10^{-14}
\\
\\
\end{aligned}
$$
We can do the same thing for division and obtain:
$$
\begin{aligned}
\\
    | \texttt{fl}(x/100) - x/100 |
        &= |x|| \delta |/100 \leq |x| 10^{-18}
\\
\\
\end{aligned}
$$
Now, these are both _relative_ bounds, in the sense that $|x|$ itself appears in the bound. 

However we can scope out what sort of values are reasonable here. Let's say we're multiplying a `double` that represents dollars by 100 to get it's cents value. To get appreciable errors (in cents) we would have to have a bound of order $O(1)$, which means $x \sim 10^{14}$! We're never going to work with financial numbers so large on a daily basis. For an amount like 1,000,000,000 (10,000,000USD) we still have about 5 digits of accuracy in the sense that 
$$
\begin{aligned}
\\
    | \texttt{fl}(100x) - 100x | &\leq 10^{-5}
\\
\\
\end{aligned}
$$
While we still have to exercise some care with rounding vs truncation, the `double` cents value of a `double` in dollars is going to be exceptionally precise even for very large amounts. 

What about dividing? It should be obvious from the bound that we have even less to worry about here. We will be dividing `double` values in cents by 100 to get `double` values in dollars. We can even convert a cent-Bezos ($x \sim O(10^{13})$) to dollars without an appreciable error (the bound is still $10^{-5}$). For 10,000,000USD we have about 11 digits of accuracy: 
$$
\begin{aligned}
\\
    | \texttt{fl}(x/100) - x/100 | &\leq 10^{-11}
\\
\\
\end{aligned}
$$

So these bounds are for the conversions alone, not the actual process of conversion, accumulation of error through repeated operations, and re-conversion. We might model this as 
$$
\begin{aligned}
\\
    x \mapsto x/100 \mapsto F(x/100) \mapsto 100 F(x/100)
\\
\\
\end{aligned}
$$
The full scope is complicated enough that it's not worth a detailed analysis, but rather exploration through examples. We can analyze conversion into dollars and re-conversion to cents: 
$$
\begin{aligned}
\\
    \texttt{fl}(100\times\texttt{fl}(x/100))
        = \texttt{fl}(100 \times (x(1+\delta_1)/100))
        = (100x/100) (1+\delta_1)(1+\delta_2)
        = x(1+\delta_1+\delta_2+\delta_1\delta_2)
        \approx x(1+2\delta)
\\
\\
\end{aligned}
$$
where $|\delta| \leq u$, of course. So we get more error, but not appreciably more error because the $\delta = O(u)$ factor is simply doubled. 

We could _assume_ a relative bound on the error of the programmatic implementation of a mapping $F$ and get a bit further. For example, suppose that $F(x/100(1+\delta)) \approx F(x/100)(1+\delta)$ (relative errors on the order of the unit roundoff in the inputs to $F$ transfer to the same order relative errors in it's values), and that $\texttt{F}(x) = F(x)(1+\eta)$ for some relative error $\eta$ (which we won't bound yet), where $\texttt{F}$ is the prgrammed version of $F$. Then
$$
\begin{aligned}
\\
    \texttt{fl}\Big(100 \times \texttt{F}\big(\texttt{fl}(x/100)\big)\Big)
        &= \texttt{fl}\Big( 100 \times \texttt{F}\big(x/100(1+\delta_1)\big)\Big) \\
        &= \texttt{fl}\Big( 100 \times F\big(x/100(1+\delta_1)\big)(1+\eta) \Big) \\
        &= \texttt{fl}\Big( 100 \times F\big(x/100\big)(1+\delta_1)(1+\eta) \Big) \\
        &= 100 F\big(x/100\big)(1+\delta_2)(1+\eta)(1+\delta_1) \\
        &\approx 100 F\big(x/100\big)(1+2\delta)(1+\eta) \\
        &\approx 100 F\big(x/100\big)(1+O(|\eta|))
\\
\\
\end{aligned}
$$
Note we've reduced this to a leading order bound in the magnitude of $\eta$. Very many repeated, possibly noisy operations on the input could greatly increase the error in the result. But it has relatively nothing to do with the conversions into or out of "dollars"; it's all related to the noisy operations in between. 

Even that isn't a complete model though. Really we are considering 
$$
\begin{aligned}
\\
    x_1,\dotsc,x_N \mapsto x_1/100,\dotsc,x_N/100 
        \mapsto F(x_1/100,\dotsc,x_N/100) \mapsto 100 F(x_1/100,\dotsc,x_N/100)
\\
\\
\end{aligned}
$$
where it might not make sense to try to describe the relative error bounds in a programmatic implementation $\texttt{F}$ of $F$. Here, as shown below for sums, the accumulation of tiny errors in each of the converted inputs can be significant in greatly increasing errors past $u$, but maybe not so great as to mean anything for out units of concern. 

### A Worst-Case Sufficient Analysis

Ok, now we can consider the second bound for sums of floating point numbers. Our `double` calculations are _guaranteed_ to be accurate _to the cents place_ if
$$
\begin{aligned}
\\
    | E_N | < 0.5 = 5\times 10^{-1} \text{ in cents }
    \quad\quad
    | E_N | < 0.005 = 5\times 10^{-3} \text{ in dollars }
\\
\\
\end{aligned}
$$
where "in cents" or "in dollars" refers to the units used in storing the amounts in `double`s. 

Let's pause and make sure this is correct. We know 
$$
\begin{aligned}
\\
    \texttt{fl}\left(\sum_n x_n\right)
        = \sum_n x_n + E_N
\\
\\
\end{aligned}
$$
(This is just the definition of $E_N$.) We presume every $x_n$ (and thus $\sum_n x_n$) is an integer, and will just `round` the left value when dealing with `double`s in cent units. Any decimal part of $\texttt{fl}\left(\sum_n x_n\right)$ comes from the error $E_N$. We also expect these errors to be small, just decimals (or we're in _really_ bad shape). If $|E_N| < 0.5$, then 
$$
\begin{aligned}
\\
    \sum_n x_n - 0.5 < \sum_n x_n + E_N < \sum_n x_n + 0.5
\\
\\
\end{aligned}
$$
and thus
$$
\begin{aligned}
\\
    \texttt{round}\left( \texttt{fl}\left(\sum_n x_n\right) \right)
        = \texttt{round}\left( \sum_n x_n + E_N \right)
        = \sum_n x_n
\\
\\
\end{aligned}
$$
by the normal rounding rule "up if the decimal is $\geq 0.5$". 

If we have dollar-`double`s, we actually round 
$$
\begin{aligned}
\\
    100 \times \texttt{fl}\left(\sum_n \left( \frac{x_n}{100} \right)\right)
        = 100 \sum_n \left( \frac{x_n}{100} \right) + 100 E_N
        = \sum_n x_n + 100 E_N
\\
\\
\end{aligned}
$$
So if $100 E_N < 0.5$, that is $E_N < 5 \times 10^{-3}$, then again 
$$
\begin{aligned}
\\
    \sum_n x_n - 0.5 < \sum_n x_n + 100E_N < \sum_n x_n + 0.5
\\
\\
\end{aligned}
$$
and the rounded value is, again, correct. 

Returning to our analysis, _sufficient_ (but probably not even close to necessary) conditions are
$$
\begin{aligned}
\\
    (N-1)u \sum_n | x_n | < 5\times 10^{-1} \text{ (cents)}
    \quad\quad
    (N-1)u \sum_n | x_n | < 5\times 10^{-3} \text{ (dollars)}
\\
\\
\end{aligned}
$$
(dropping the $O(u^2)$ "remainder", as it won't be relevant). We've chosen numbers $x_n$ such that, in dollars (because we want the `double` decimals accurate to cents), satisfy $|x_n| \leq 10^5$. Then, compounding our very sufficient condition further with other loose bounds, we have 
$$
\begin{aligned}
\\
    \text{cents: }\quad
        &(N-1)u \sum_n | x_n | 
        \leq 10^7 (N-1)N u 
        \leq 10^{-9} N^2
         \\
    \text{dollars:}\quad
        &(N-1)u \sum_n | x_n | 
        \leq 10^5 (N-1)N u 
        \leq 10^{-11} N^2
\\
\\
\end{aligned}
$$
Obviously if the right-hand sides are small enough, we have a bound on the error guaranteeing correctness: 
$$
\begin{aligned}
\\
    \text{cents: }\quad
        &10^{-9} N^2 < 5\times10^{-1}
        \quad\iff\quad
        N < \sqrt{5}\times10^{4} \approx 22360
         \\
    \text{dollars:}\quad
        &10^{-11} N^2 < 5\times10^{-3}
        \quad\iff\quad
        N < \sqrt{5}\times10^{4} \approx 22360
\\
\\
\end{aligned}
$$
At least in this analysis, our factors of 100 cancel and we get the same result. 

The result is that, even in this loosest of worst case analyses with the simplest summation strategy, the (IEEE) `double` calculation is _guaranteed_ to be cent-accurate for sums of less than $22,360$ items of the magnitude we assume. We assume pretty large magnitude amounts, on average; reducing this size to a more reasonable level like 100USD or 1000USD would increase $N$. Specifically, if the largest $x_n$ is like $M = 10^m$ in dollars, 
$$
\begin{aligned}
\\
    \text{cents: }\quad
        &(N-1)u \sum_n | x_n | 
        \leq 10^{-(14-m)} N^2
         \\
    \text{dollars:}\quad
        &(N-1)u \sum_n | x_n | 
        \leq 10^{-(16-m)} N^2
\\
\\
\end{aligned}
$$
and our sufficient condition is
$$
\begin{aligned}
\\
    \text{cents: }\quad
        &10^{-(14-m)} N^2 < 5\times10^{-1}
        \quad\iff\quad
        N < \sqrt{5}\times10^{6.5-m/2}
         \\
    \text{dollars:}\quad
        &10^{-(16-m)} N^2 < 5\times10^{-3}
        \quad\iff\quad
        N < \sqrt{5}\times10^{6.5-m/2}
\\
\\
\end{aligned}
$$
For $m = 3$ and amounts below 1,000USD, these bounds are about $N < 223,606$; for $m = 2$, $N < 2,236,067$. 

### A Technicality

Technically we haven't included the errors in converting to dollars. If we want to handwave around this, we can make an argument perhaps like the following: The error we care about is most like
$$
\begin{aligned}
\\
    E_N &= \left| \; \sum_n \frac{c_n}{100} - \texttt{fl}\left(\sum_n \texttt{fl}\left(\frac{c_n}{100}\right)\right) \; \right| \\
        &= \left| \; \sum_n \frac{c_n}{100} - \texttt{fl}\left(\sum_n \frac{c_n}{100}(1+\delta_n)\right) \; \right|
\\
\\
\end{aligned}
$$
(We should probably also add in the float conversions make to cents.) This is not what is bounded in the sum above, but we can turn it into something like that as follows: 
$$
\begin{aligned}
\\
    \left| \; | E_N | - \Bigg| \sum_n \frac{c_n}{100}\delta_n \Bigg| \; \right|
        &\leq \left| \; E_N - \left( - \sum_n \frac{c_n}{100}\delta_n \right) \; \right|
        = \left| \; E_N + \sum_n \frac{c_n}{100}\delta_n \; \right| \\
        &= \left| \; \sum_n \frac{c_n}{100}(1+\delta_n) - \texttt{fl}\left(\sum_n \frac{c_n}{100}(1+\delta_n)\right) \; \right|  \\
        &\leq (N-1)u \sum_n \left | \frac{c_n}{100} \right ||1+\delta_n| \\
        &\leq (N-1)u \sum_n \left | \frac{c_n}{100} \right | 
                + (N-1)u \sum_n \left | \frac{c_n}{100} \right | | \delta_n| \\
        &= (N-1)u \Bigg( 1 + \sum_n w_n | \delta_n| \Bigg) \sum_n \left | \frac{c_n}{100} \right |
        \quad\quad
        w_n = \frac{| c_n/100 |}{\sum_m | c_m/100 | }
        \\
        &= (N-1)u \sum_n \left | \frac{c_n}{100} \right | + O(u^2) \\
        &\approx (N-1)u \sum_n \left | \frac{c_n}{100} \right |
\\
\\
\end{aligned}
$$
What this says about the error magnitude $|E_N|$ we care about is
$$
\begin{aligned}
\\
    |E_N| = \Bigg| \sum_n \frac{c_n}{100}\delta_n \Bigg| + \triangle
    \quad\quad
    |\triangle| \leq (N-1)u \sum_n \left | \frac{c_n}{100} \right |
\\
\\
\end{aligned}
$$
Going way out on a limb, let's say 
$$
\begin{aligned}
\\
    |E_N| = 10^{-( 18 - \log_{10}N - \log_{10}\mathbb{E}c )} + \triangle
    \quad\quad
    |\triangle| \leq 10^{-( 18-\log_{10}\mathbb{E}c - 2\log_{10}N)}
\\
\\
\end{aligned}
$$
The dominant term is still $|\triangle|$, because of the factor $2\log_{10}N$. 

For example, in our code $\log_{10}\mathbb{E}c = 6$ (which is actually kind of huge), and 
$$
\begin{aligned}
\\
    |E_N| = 10^{-( 12 - \log_{10}N )} + \triangle
    \quad\quad
    |\triangle| \leq 10^{-2 ( 6 - \log_{10}N )}
\\
\\
\end{aligned}
$$
Then if $\log_{10}N \sim 5$, we have 
$$
\begin{aligned}
\\
    |E_N| = 10^{-7} + \triangle
    \quad\quad
    |\triangle| \leq 10^{-2}
\\
\\
\end{aligned}
$$

### Improving the Bounds

It's possible to scream "CENTRAL LIMIT THEOREM" at the sky and get some slightly better bounds, but not really by much. Constant factors, instead of orders of magnitude. Maybe I'll work on refining some of that later, but for now contact me if you're interested. 

### Pairwise Summation

There are other ways to sum a bunch of numbers, one of them being _pairwise summation_ (Chapter 4 of Higham again). Basically, sum pairs of numbers, then sum those sums, then sum _those_ sums, and so on until the summation is done. This method is easy enough to implement as has a much tighter error bound: 
$$
\begin{aligned}
\\
    | E_N | \leq \frac{\log_2N \; u}{1-\log_2N \; u} \sum_n |x_n|
\\
\\
\end{aligned}
$$
The $\log_2N$ factor suggests much smaller errors. But let's suppose $\log_2N \; u \approx 0$ (which is fair for any reasonable $N$ we could use) and redo our worst-case analysis: 
$$
\begin{aligned}
\\
    \frac{\log_2N \; u}{1-\log_2N \; u} \sum_n |x_n|
        \approx \log_2N \; u \sum_n |x_n|
        \leq 10^5 N \log_2N \; u 
        = \frac{N \log_2N}{10^{11}}
\\
\\
\end{aligned}
$$
For what values of $N$ is the RHS guaranteed to be less than $0.005$ (focusing on dollars, since it'll work out the same anyway)? Clearly
$$
\begin{aligned}
\\
    \frac{N \log_2N}{10^{11}} < 5 \times 10^{-3}
    \quad\iff\quad
    N \log_2N < 5 \times 10^{8}
\\
\\
\end{aligned}
$$
With some transformations ($\log$ transforms and Newton's method), we can estimate 
$$
\begin{aligned}
\\
    N \log_2N < 5 \times 10^{8}
    \quad\iff\quad 
    N < 20,580,553
\\
\\
\end{aligned}
$$
So if $N$ (the number of amounts to sum) is under about 20 million, we can use pairwise summation and rest assured that the value obtained using `double`s will be correct to the cents place.  

The scaling here is much different than for $N^2$ though. Generalizing to $|x_n| \leq 10^m$ (in dollars), 
$$
\begin{aligned}
\\
    \log_2N \; u \sum_n |x_n|
        \leq N \log_2 N 10^{-(16-m)}
        < 5 \times 10^{-3}
        \quad\iff\quad 
        N \log_2 N < 5 \times 10^{13-m}
\\
\\
\end{aligned}
$$
which for $m = 3$ is about $N < 1,633,686,624$. That is, for amounts not less than thousands (probably also "typically" thousands) of dollars, $N$ can be as much as a billion and a half and still be _guaranteed_ accuracy with pairwise summation. 

Let's try it! The code `pairwise.c` has an implementation of the pairwise summation strategy, but is otherwise simpler in that we only store the `long int`s and `double`s. Otherwise we can generate and analyze results in the same way. 

In [242]:
!gcc -O3 pairwise.c -o pairwise

libc++abi.dylib: terminating with uncaught exception of type std::runtime_error: Couldn't close file


In [246]:
# !./pairwise 100 10000 pairwise-100-10000.csv
df = pd.read_csv( 'pairwise-100-10000.csv' , header=None , names=['trial','op','f_c','f_d','d_c','d_d'] )
df[['op','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

f_c       f_d       d_c       d_d     
   mean  std mean  std mean  std mean  std
op                                        
+     1  0.0    1  0.0    1  0.0    1  0.0
-     1  0.0    1  0.0    1  0.0    1  0.0
?     1  0.0    1  0.0    1  0.0    1  0.0

In [247]:
# !./pairwise 1000 10000 pairwise-1000-10000.csv
df = pd.read_csv( 'pairwise-1000-10000.csv' , header=None , names=['trial','op','f_c','f_d','d_c','d_d'] )
df[['op','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

f_c          f_d            d_c       d_d     
   mean  std    mean       std mean  std mean  std
op                                                
+     1  0.0  0.8165  0.387095    1  0.0    1  0.0
-     1  0.0  0.8165  0.387095    1  0.0    1  0.0
?     1  0.0  1.0000  0.000000    1  0.0    1  0.0

In [248]:
# !./pairwise 10000 10000 pairwise-10000-10000.csv
df = pd.read_csv( 'pairwise-10000-10000.csv' , header=None , names=['trial','op','f_c','f_d','d_c','d_d'] )
df[['op','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

f_c              f_d            d_c       d_d     
      mean      std    mean       std mean  std mean  std
op                                                       
+   0.0965  0.29529  0.0950  0.293230    1  0.0    1  0.0
-   0.0965  0.29529  0.0950  0.293230    1  0.0    1  0.0
?   1.0000  0.00000  0.9599  0.196204    1  0.0    1  0.0

In [254]:
# !./pairwise 100000 10000 pairwise-100000-10000.csv
df = pd.read_csv( 'pairwise-100000-10000.csv' , header=None , names=['trial','op','f_c','f_d','d_c','d_d'] )
df[['op','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

f_c               f_d            d_c       d_d     
      mean       std    mean       std mean  std mean  std
op                                                        
+   0.0093  0.095992  0.0081  0.089639    1  0.0    1  0.0
-   0.0093  0.095992  0.0081  0.089639    1  0.0    1  0.0
?   0.9829  0.129651  0.4528  0.497792    1  0.0    1  0.0

In [249]:
# !./pairwise 1000000 10000 pairwise-1000000-10000.csv
df = pd.read_csv( 'pairwise-1000000-10000.csv' , header=None , names=['trial','op','f_c','f_d','d_c','d_d'] )
df[['op','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

libc++abi.dylib: terminating with uncaught exception of type std::runtime_error: Couldn't close file


f_c                 f_d            d_c       d_d     
        mean       std      mean       std mean  std mean  std
op                                                            
+   0.000882  0.029692  0.000882  0.029692  1.0  0.0  1.0  0.0
-   0.000882  0.029692  0.000882  0.029692  1.0  0.0  1.0  0.0
?   0.491619  0.499985  0.137186  0.344081  1.0  0.0  1.0  0.0

In [250]:
# !./pairwise 10000000 10000 pairwise-10000000-10000.csv
df = pd.read_csv( 'pairwise-10000000-10000.csv' , header=None , names=['trial','op','d_c','d_d'] )
df[['op','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] )

libc++abi.dylib: terminating with uncaught exception of type std::runtime_error: Couldn't close file


d_c       d_d     
   mean  std mean  std
op                    
+     1  0.0    1  0.0
-     1  0.0    1  0.0
?     1  0.0    1  0.0

So, as predicted, pairwise summation has improved the correctness of using `double`s to represent amounts, whether in cent or dollar units. Even with sums of 10M values, `double`s give the correct results when rounded to the cents place. 

# Learnings

What have we learned? 

Mainly, I think, we've learned that 4 bytes is not enough. `int`s and `float`s perform very poorly in accurate sums for amounts following our distribution. 

However using 8 byte integers (`long int`s) representing cents give us perfect answers for addition and subtraction because of the sheer size of their range relative to the calculations we probably need to perform, particularly sums of (possibly signed) integers representing monetary values (in cents). 

`double`s, also 8 bytes but subject to roundoff error, are also probably fine too. We saw no errors in tests using `double`s pegged to cents, but some errors for "very large" sums of (millions of) uniformly-signed amounts pegged to dollars, but not for mixed sign sums. Moreover, if we trust the gurus of numerical analysis in the presence of round-off errors (which we should), there are actual theoretical guarantees of the correctness of using `double`s for tens of thousands of summands with naive summation code and at least tens of millions of summands, if not billions, with smarter code. Both theoretical conclusions are justified by numerical experiments. 

In [265]:
for N in [100,1000,10000,100000,1000000,10000000]: 
    df = pd.read_csv( f'seq-bins-{N}-10000.csv' , header=None , names=['trial','op','i_c','f_c','f_d','d_c','d_d'] )
    print( 100.0*df[['op','i_c','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] ) )

      i_c         f_c          f_d              d_c         d_d     
     mean  std   mean  std    mean       std   mean  std   mean  std
op                                                                  
+   100.0  0.0  100.0  0.0   99.33  8.158294  100.0  0.0  100.0  0.0
-   100.0  0.0  100.0  0.0   99.33  8.158294  100.0  0.0  100.0  0.0
?   100.0  0.0  100.0  0.0  100.00  0.000000  100.0  0.0  100.0  0.0
      i_c         f_c         f_d               d_c         d_d     
     mean  std   mean  std   mean        std   mean  std   mean  std
op                                                                  
+   100.0  0.0  100.0  0.0   8.43  27.785109  100.0  0.0  100.0  0.0
-   100.0  0.0  100.0  0.0   8.43  27.785109  100.0  0.0  100.0  0.0
?   100.0  0.0  100.0  0.0  82.05  38.378963  100.0  0.0  100.0  0.0
      i_c          f_c             f_d               d_c         d_d     
     mean  std    mean      std   mean        std   mean  std   mean  std
op                      

In [269]:
for N in [100,1000,10000,100000,1000000,10000000]: 
    df = pd.read_csv( f'pairwise-{N}-10000.csv' , header=None , names=['trial','op','f_c','f_d','d_c','d_d'] )
    print( 100.0*df[['op','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] ) )

      f_c              f_d               d_c         d_d     
     mean        std  mean        std   mean  std   mean  std
op                                                           
+    2.04  14.137128  2.00  14.000700  100.0  0.0  100.0  0.0
-    2.04  14.137128  2.00  14.000700  100.0  0.0  100.0  0.0
?   12.03  32.532829  9.87  29.827371  100.0  0.0  100.0  0.0
     f_c              f_d               d_c         d_d     
    mean        std  mean        std   mean  std   mean  std
op                                                          
+   0.12   3.462196  0.21   4.577990  100.0  0.0  100.0  0.0
-   0.12   3.462196  0.21   4.577990  100.0  0.0  100.0  0.0
?   2.72  16.267393  2.63  16.003397  100.0  0.0  100.0  0.0
     f_c           f_d              d_c         d_d     
    mean     std  mean       std   mean  std   mean  std
op                                                      
+   0.04  1.9997  0.00  0.000000  100.0  0.0  100.0  0.0
-   0.04  1.9997  0.00  0.000000  

In [267]:
for N in [100,1000,10000,100000,1000000,10000000]: 
    df = pd.read_csv( f'bins-pairwise-{N}-10000.csv' , header=None , names=['trial','op','f_c','f_d','d_c','d_d'] )
    print( 100.0*df[['op','f_c','f_d','d_c','d_d']].groupby('op').agg( [ 'mean' , 'std' ] ) )

      f_c         f_d         d_c         d_d     
     mean  std   mean  std   mean  std   mean  std
op                                                
+   100.0  0.0  100.0  0.0  100.0  0.0  100.0  0.0
-   100.0  0.0  100.0  0.0  100.0  0.0  100.0  0.0
?   100.0  0.0  100.0  0.0  100.0  0.0  100.0  0.0
      f_c          f_d               d_c         d_d     
     mean  std    mean        std   mean  std   mean  std
op                                                       
+   100.0  0.0   81.65  38.709525  100.0  0.0  100.0  0.0
-   100.0  0.0   81.65  38.709525  100.0  0.0  100.0  0.0
?   100.0  0.0  100.00   0.000000  100.0  0.0  100.0  0.0
       f_c               f_d               d_c         d_d     
      mean        std   mean        std   mean  std   mean  std
op                                                             
+     9.65  29.529048   9.50  29.322960  100.0  0.0  100.0  0.0
-     9.65  29.529048   9.50  29.322960  100.0  0.0  100.0  0.0
?   100.00   0.000000  95.

In [288]:
for N in [100,1000,10000,100000]: # ,1000000,10000000]: 
    df = pd.read_csv( f'blocksums-{N}-10000.csv' , header=None , names=['trial',f'N={N}','f_c','f_d'] )
    print( 100.0*df[[f'N={N}','f_c','f_d']].groupby(f'N={N}').agg( [ 'mean' , 'std' ] ) )

         f_c         f_d     
        mean  std   mean  std
N=100                        
+      100.0  0.0  100.0  0.0
-      100.0  0.0  100.0  0.0
?      100.0  0.0  100.0  0.0
          f_c         f_d     
         mean  std   mean  std
N=1000                        
+       100.0  0.0  100.0  0.0
-       100.0  0.0  100.0  0.0
?       100.0  0.0  100.0  0.0
           f_c         f_d     
          mean  std   mean  std
N=10000                        
+        100.0  0.0  100.0  0.0
-        100.0  0.0  100.0  0.0
?        100.0  0.0  100.0  0.0
             f_c                f_d           
            mean        std    mean        std
N=100000                                      
+          49.66  50.001344   37.81  48.493701
-          49.66  50.001344   37.81  48.493701
?         100.00   0.000000  100.00   0.000000


In [292]:
# N log2 N - 5 x 10^{ 7 - log10 mu }
# d/dN = log2 N + 1/log(2)

def solve( mu , N0=10 ): 
    c = 5 * 10**( 7 - log10(mu) )
    d = 1.0/log(2)
    N = N0
    F = N * log2(N) - c
    while abs(F) > 1e-6:
        D = log2(N) + d
        s = - F / D
        N = N + s
        F = N * log2(N) - c
    return N

mus , Ns = [10000,50000,100000,500000,1000000] , []
for mu in mus:
    Ns.append( solve( mu , N0=Ns[-1] if len(Ns) > 0 else 10 ) )
[ (N,log2(N)) for N in Ns ]

[(549.3543663360359, 9.101593263648583),
 (140.2216669915382, 7.131565480963959),
 (79.25827396991411, 6.308489644553096),
 (22.32008145855597, 4.480270387271322),
 (13.366874456930422, 3.7405902579526114)]